## Getting Album Recommendations from P4K

**Read User Library, sort on an audio feature and threshold, and find 2017 album recommendations from P4K based on your weighted audio features (by standard deviations and mean)**

Requirements:

Have 'sp-&-p4k_combined-2017-utf8sig-final.csv' and 'our_tracks.csv' in same directory as NB.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS


pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline


In [ ]:
# Optional: Only if you don't have this.
pip install wordcloud

In [2]:
combined_reviews_fname = 'sp-&-p4k_combined-2017-utf8sig-final.csv'

sp_p4k_2017_df = pd.read_csv(combined_reviews_fname, index_col = 0)

sp_p4k_2017_df.tail()

,album_uri,album,album_artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,valence,popularity,duration_ms,is_explicit,artist,best,date,genre,review,score
1058,spotify:album:7z7hGOK5ZvGQ2rCp9TyMHc,Happy Together,Mega Bog,0.4337,0.4270,0.6402,0.2764,0.1942,-7.2556,0.0519,...,0.4747,5.7778,1638435,0,Mega Bog,0,February 6 2017,Experimental,Mega Bog is the malleable rock band mastermind...,7.4000
1059,spotify:album:7zJ6TVgvzdV9HPE8KA2a7R,Hate Songs EP,Suicideyear,0.1792,0.5723,0.6933,0.5878,0.1883,-6.0307,0.0513,...,0.1758,9.5000,1548171,0,Suicideyear,0,July 28 2017,Electronic,Stepping away from his habitual trap beats and...,7.3000
1060,spotify:album:7zg9ZSOQtYLjWnvSawflg6,Born Again,The Notorious B.I.G.,0.1496,0.6924,0.7748,0.0038,0.2771,-5.6680,0.3424,...,0.6352,44.8824,4294841,17,The Notorious B.I.G.,0,March 9 2017,Rap,The half-baked 1999 release Born Again reeks o...,6.0000
1061,spotify:album:7zo4kiBFeJ6fTRJLTKo7XU,Belladonna of Sadness,Alexandra Savior,0.0671,0.5226,0.6552,0.0029,0.2239,-7.2379,0.0403,...,0.5956,41.2000,2248784,1,Alexandra Savior,0,April 19 2017,Pop/R&B,The young American singer launches her career ...,6.0000
1062,spotify:album:7zy6BF4J1cdLVZaPoTbLa7,All Blue,G Perico,0.1640,0.8337,0.6395,0.0000,0.1961,-6.9567,0.3181,...,0.5325,22.1818,1774000,11,G Perico,0,May 5 2017,Rap,While nodding to West Coast legends of the ’90...,7.9000


In [3]:
def plotWordCloud(df_word_cloud):
    """
    Input any df with reviews in it. Generates word clout plot for their reviews.
    """
    comment_words = ''
    additional_stopwords = set(("song", "album", "record", "music", "songs", "track"))
    stopwords =  set(STOPWORDS).union(additional_stopwords)

    df_word_cloud = df_word_cloud.review

    # iterate through the csv file
    for val in df_word_cloud:

        # typecaste each val to string
        val = str(val)

        # split the value
        tokens = val.split()
        tokens = [token for token in tokens if len(token) > 1]

        # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()

        comment_words += " ".join(tokens)+" "

    wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10, collocations=False).generate(comment_words)

    # plot the WordCloud image                       
    plt.figure(figsize = (10, 10), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)

    plt.show()

## Add column to DF to indicate if album is in Group 1, 2, 3, 4

Group 1: "favorites"
- above average popularity, above average score

Group 2: "underrated"
- below average popularity, above average score

Group 3: "overrated"
- above average popularity, below average score

Group 4: "ignorables"
- below average popularity, below average score

In [4]:
mean_score_allData = sp_p4k_2017_df.score.mean()
mean_pop_allData = sp_p4k_2017_df.popularity.mean()

def label_group(row):
   if (row['popularity'] > mean_pop_allData) and (row['score'] > mean_score_allData) :
      return 'favorites'
   if (row['popularity'] <= mean_pop_allData) and (row['score'] > mean_score_allData) :
      return 'underrated'
   if (row['popularity'] > mean_pop_allData) and (row['score'] <= mean_score_allData) :
      return 'overrated'
   else:
      return 'ignorables'

sp_p4k_2017_df['group_label'] = sp_p4k_2017_df.apply(lambda row: label_group(row), axis=1)
sp_p4k_2017_df.head(5)

,album_uri,album,album_artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,popularity,duration_ms,is_explicit,artist,best,date,genre,review,score,group_label
0,spotify:album:00LS6GJwb5ZPTKS8NMAT8x,Ultramega OK,Soundgarden,0.0167,0.3555,0.8228,0.2762,0.1938,-8.2745,0.0634,...,23.5263,4224568,0,Soundgarden,0,March 20 2017,Rock,"Originally released in 1988 on SST, Soundgarde...",7.3000,underrated
1,spotify:album:00MAXeszCotk3g9q8KYJlZ,Elastica,Elastica,0.0132,0.5434,0.7989,0.3546,0.1419,-5.6124,0.0444,...,27.6429,2186449,1,Elastica,0,September 24 2017,Rock,"With its pithy, searing songs about sex, group...",8.5000,favorites
2,spotify:album:00RnMn3ikeVB6gdOMZ3mwQ,No Mountains in Manhattan,Wiki,0.4477,0.5967,0.7189,0.0005,0.3150,-6.6996,0.3737,...,18.4000,3191839,15,Wiki,0,August 30 2017,Rap,Wiki’s new album is an acidic and vivid manife...,8.1000,underrated
3,spotify:album:02ER13KaisZo5CG2BydCWn,Americana,Ray Davies,0.4588,0.5249,0.4491,0.0010,0.1896,-12.1996,0.0432,...,11.6667,3511114,0,Ray Davies,0,April 18 2017,Rock,The Kinks legend uses Americana to blaze a pat...,7.3000,underrated
4,spotify:album:02Lx6I5EsUDFIdnS3pSNNT,Rosewood Almanac,Will Stratton,0.6693,0.4786,0.3263,0.0458,0.1885,-12.1106,0.0342,...,7.6667,1775810,0,Will Stratton,0,May 22 2017,Folk/Country,"With pastoral folk roots, New York singer-song...",7.0000,ignorables


## Getting Recommendations based on vector input

For our personal datasets, we can choose an audio feature, give a threshold value and specify above or below for direction. We can generate a vector of other audio features for our libraries based on this threshold. We want the audio feature, average value, and weight (assumed = 1). The weights can be changed based on the inverse of the variance of the audio features, and their relative %.

Then we get the subset of p4k data with this audio feature threshold met. Lets say valence above 0.3 for example. 

On this subset, we will apply a row function to create the distance between our library vector and the album's audio features. The distance will only use the audio features specified by our library vector, and will use the weights specified for each audio feature.

Each album will have a distance added as a variable. Sort by shortest distance to find the closest recommended albums for the user.

In [6]:
def getAlbumDistance(row, library_vector: dict):
    """
    Inputs:
    row: the album row from data frame
    library_vector: dictionary of audio feature keys, and values as average_value, weight of each key
    """
    distance = 0
    for audio_feature in library_vector:
        temp_dist = library_vector[audio_feature]['weight'] * np.power((row[audio_feature] - library_vector[audio_feature]['average_value']), 2)
        distance = distance + temp_dist
    return np.sqrt(distance)

In [7]:
def P4K_Recommended(by_audio_feature: str, threshold: float, above_threshold: bool, library_vector: dict, P4K_DF):
    """
    by_audio_feature: name of audio feature to filter pitchfork df by
    """
    # get a copy of the p4k DF based on the audio feature
    if above_threshold:
        focused_P4K_DF = P4K_DF[P4K_DF[by_audio_feature] > threshold].copy()
    else:
        focused_P4K_DF = P4K_DF[P4K_DF[by_audio_feature] <= threshold].copy()
    
    focused_P4K_DF['distance'] = focused_P4K_DF.apply(lambda row: getAlbumDistance(row, library_vector), axis=1)
    return focused_P4K_DF.sort_values('distance', ascending=True)

## Library Data to get our album recommendations

In [8]:
# load in our tracks df
our_tracks_df = pd.read_csv("our_tracks.csv")

# group by users
our_tracks_by_users = our_tracks_df.groupby('username')


Get a user's tracks df. Using mine for testing.

In [9]:
anand_df = our_tracks_by_users.get_group('Anand')

Get some threshold values for our audio feature to stratify on. We will look at valence (low and high) and energy (low, high) in our study.

In [10]:
user_df = anand_df

user_low_valence_thres = user_df['valence'].quantile(0.25)
user_high_valence_thres = user_df['valence'].quantile(0.75)

user_low_energy_thres = user_df['energy'].quantile(0.25)
user_high_energy_thres = user_df['energy'].quantile(0.75)

In [11]:
def get_library_vector(user_df, audio_feature: str, threshold: float, is_above: bool, is_weighted: bool):
    """
    Returns the library vector and weights for the audio features
    """
    # get the filtered df for the user
    if is_above:
        filtered_user_df = user_df[user_df[audio_feature] > threshold]
    else:
        filtered_user_df = user_df[user_df[audio_feature] <= threshold]
    
    # generate the vector for only the audio_features
    mean_srs = filtered_user_df.mean()
    mean_srs = mean_srs.drop(labels=[audio_feature, 'duration_ms', 'is_explicit', 'key', 'mode'])
    std_srs = filtered_user_df.std()
    std_srs = std_srs.drop(labels=[audio_feature, 'duration_ms', 'is_explicit', 'key', 'mode'])
    
    # generate weight series
    temp = std_srs/mean_srs
    temp2 = np.abs(np.power(temp, -1))
    weights_srs = (temp2/temp2.sum())*100
    
    library_user_dict = {}
    
    for audio_feat in mean_srs.index:
        library_user_dict[audio_feat] = {}
        library_user_dict[audio_feat]['average_value'] = mean_srs[audio_feat]
        
        if is_weighted:
            library_user_dict[audio_feat]['weight'] = weights_srs[audio_feat]
        else:
            library_user_dict[audio_feat]['weight'] = 1
    return library_user_dict
    

High Valence Pitchfork Album Recommendations:

In [12]:
# generate vector based on user library
user_high_valence_dict = get_library_vector(user_df, 'valence', user_high_valence_thres, True, True)
print(user_high_valence_dict)

# get closest matches from pitchfork album reviews
user_recs_p4k_high_valence = P4K_Recommended('valence', user_high_valence_thres, True, user_high_valence_dict, sp_p4k_2017_df)
user_recs_p4k_high_valence.head(10)

{'popularity': {'average_value': 28.59, 'weight': 7.411666424041666}, 'danceability': {'average_value': 0.6163599999999998, 'weight': 21.24018473140925}, 'energy': {'average_value': 0.6533500000000003, 'weight': 15.528607521740945}, 'loudness': {'average_value': -9.5753, 'weight': 11.302087275142995}, 'speechiness': {'average_value': 0.07739200000000006, 'weight': 4.798867066988189}, 'acousticness': {'average_value': 0.3201709399999999, 'weight': 4.7609228543446696}, 'instrumentalness': {'average_value': 0.44287093480000017, 'weight': 5.607124793850626}, 'liveness': {'average_value': 0.17945500000000003, 'weight': 5.84324581067509}, 'tempo': {'average_value': 128.59617000000003, 'weight': 23.507293521806556}}


,album_uri,album,album_artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,duration_ms,is_explicit,artist,best,date,genre,review,score,group_label,distance
600,spotify:album:4HuzPUi36WvuyGWKUcQEkt,Lifestyles of the Laptop Café,The Other People Place,0.0240,0.7437,0.4385,0.8565,0.0992,-11.6031,0.0824,...,3113530,0,The Other People Place,1,February 13 2017,Electronic,Best new reissue The only record by Drexciya’s...,8.8000,favorites,8.3838
229,spotify:album:1jmReMlhBKjUB7KXg9Pdxm,Thawing Dawn,A. Savage,0.3689,0.4760,0.4859,0.2632,0.1468,-9.4047,0.0355,...,2757123,0,A. Savage,0,October 23 2017,Rock,The debut solo LP from Andrew Savage of Parque...,7.2000,overrated,9.0084
155,spotify:album:1CVuPxNHwY5ORJ8MhjD0UB,Freedom Highway,Rhiannon Giddens,0.7223,0.6461,0.3407,0.0844,0.1275,-11.2980,0.0610,...,2726250,0,Rhiannon Giddens,0,February 23 2017,Folk/Country,Rhiannon Giddens emerges as a peerless and pow...,7.6000,favorites,10.5839
77,spotify:album:0d6ky7EGJ0DBxCsSR9R5SR,Take Flight,Maya Jane Coles,0.2233,0.7122,0.7256,0.6971,0.1646,-7.2659,0.0524,...,6436775,1,Maya Jane Coles,0,August 23 2017,Electronic,The first major release in three years from Br...,6.0000,overrated,12.0145
1,spotify:album:00MAXeszCotk3g9q8KYJlZ,Elastica,Elastica,0.0132,0.5434,0.7989,0.3546,0.1419,-5.6124,0.0444,...,2186449,1,Elastica,0,September 24 2017,Rock,"With its pithy, searing songs about sex, group...",8.5000,favorites,14.2950
759,spotify:album:5RYg5ZR0qfRebkseuqlGJj,Moonshine Freeze,This Is the Kit,0.6982,0.5047,0.4357,0.0752,0.1442,-10.3621,0.0418,...,2527090,0,This Is the Kit,0,July 26 2017,Folk/Country,"With her new record as This Is the Kit, singer...",7.3000,favorites,14.7626
319,spotify:album:2NF7kUOEaNYhCWLANV85Aj,Joli Mai,Daphni,0.1557,0.7351,0.6327,0.8608,0.1288,-11.8086,0.0544,...,3363923,0,Daphni,0,October 28 2017,Electronic,Dan Snaith’s new LP as Daphni contains unmixed...,6.8000,overrated,15.3250
403,spotify:album:2vTelM2ZV20cLPqQwfWhYa,Dots and Loops,Stereolab,0.4038,0.5759,0.5274,0.4300,0.1382,-11.0015,0.0549,...,6277297,0,Stereolab,0,July 23 2017,Experimental,Stereolab’s masterpiece fused analog with digi...,9.2000,favorites,17.2470
895,spotify:album:6VNXIYzXocTyZMNDLG88Gb,Mista Thug Isolation,Lil Ugly Mane,0.0441,0.5880,0.7539,0.0539,0.2220,-7.7724,0.2247,...,3874722,18,Lil Ugly Mane,0,April 12 2017,Rap,Noise musician turned rapper Travis Miller mad...,8.2000,favorites,17.5226
861,spotify:album:6GdJPhGu12KSZ7KdgBfye1,One Night Only,Cousin Stizz,0.3071,0.8574,0.5773,0.0018,0.1512,-5.2903,0.0998,...,2343888,12,Cousin Stizz,0,July 20 2017,Rap,"On his major label debut, the once-scrappy Bos...",6.9000,overrated,18.1016


Low Valence Pitchfork Album Recommendations:

In [13]:
# generate vector based on user library
user_low_valence_dict = get_library_vector(user_df, 'valence', user_low_valence_thres, False, True)
print(user_low_valence_dict)

# get closest matches from pitchfork album reviews
user_recs_p4k_low_valence = P4K_Recommended('valence', user_low_valence_thres, False, user_low_valence_dict, sp_p4k_2017_df)
user_recs_p4k_low_valence.head(10)

{'popularity': {'average_value': 25.91, 'weight': 8.405718712752789}, 'danceability': {'average_value': 0.30931300000000006, 'weight': 10.434289407037918}, 'energy': {'average_value': 0.33428099999999994, 'weight': 8.565520237918209}, 'loudness': {'average_value': -14.889910000000002, 'weight': 13.54049984619805}, 'speechiness': {'average_value': 0.041903, 'weight': 14.094088646865218}, 'acousticness': {'average_value': 0.5639588099999999, 'weight': 8.153175825135037}, 'instrumentalness': {'average_value': 0.5835046699999998, 'weight': 8.882428180081375}, 'liveness': {'average_value': 0.16291500000000006, 'weight': 7.254158492677626}, 'tempo': {'average_value': 109.08485, 'weight': 20.67012065133377}}


,album_uri,album,album_artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,duration_ms,is_explicit,artist,best,date,genre,review,score,group_label,distance
815,spotify:album:5n5V1pcevIOdJBF4xfuMlX,Finding People EP,Croatian Amor,0.8590,0.2340,0.2640,0.8780,0.0755,-16.5870,0.0347,...,202264,0,Croatian Amor,0,May 10 2017,Electronic,"On his new EP, Danish electronic artist Loke R...",6.3000,ignorables,28.2666
126,spotify:album:0yf8QgcSSUM9x2CRRtalwL,Grunewald,Peter Broderick,0.9792,0.2760,0.0760,0.6581,0.0969,-20.4394,0.0450,...,1381786,0,Peter Broderick,0,December 16 2016,Experimental,Composer Peter Broderick manages to make simpl...,6.7000,ignorables,33.5535
607,spotify:album:4JQ2igmQEWUihSRzWgTiCF,Narkopop,GAS,0.1794,0.2402,0.1649,0.7607,0.1264,-23.1950,0.0416,...,8996550,0,GAS,1,April 21 2017,Electronic,"Best new music After 17 years, Wolfgang Voigt ...",8.8000,underrated,38.5201
154,spotify:album:1CICQQAxvXbayxDF5jJETJ,Trumpeting Ecstasy,Full of Hell,0.0001,0.2251,0.8876,0.7574,0.4400,-4.7332,0.1098,...,1404611,0,Full of Hell,0,May 9 2017,Metal,The new album from crushers Full of Hell is an...,7.7000,favorites,39.7792
366,spotify:album:2eOMttZbB9lBc9LLMj56xj,Luciferian Towers,Godspeed You! Black Emperor,0.3765,0.1699,0.4804,0.7689,0.1454,-11.5638,0.0424,...,2634017,0,Godspeed You! Black Emperor,0,September 21 2017,Rock,The sound of Godspeed’s radical fury takes a s...,7.3000,favorites,40.1779
782,spotify:album:5brOOtl1XN2Li2IdYBDNzr,Parallels,Nosaj Thing,0.6233,0.4389,0.3529,0.7081,0.1027,-15.2556,0.0446,...,1531420,0,Nosaj Thing,0,September 18 2017,Electronic,"The new album from L.A. producer Jason Chung, ...",6.4000,ignorables,44.9642
394,spotify:album:2rWkvguXepRyMmxIe8Brx3,Balsams,Chuck Johnson,0.7917,0.1106,0.1365,0.8953,0.1041,-17.6880,0.0403,...,2481485,0,Chuck Johnson,0,June 5 2017,Experimental,"On this expressive and singular new album, Chu...",8.1000,favorites,51.4000
299,spotify:album:2A9lnshn6QxS5SmbCsCuu0,Reflections - Mojave Desert,Floating Points,0.4780,0.2496,0.3632,0.7934,0.2335,-14.9568,0.0513,...,3144274,0,Floating Points,0,June 28 2017,Electronic,Sam Shepherd and his ensemble incorporate the ...,7.3000,underrated,52.0491
592,spotify:album:4Ce7dnSqt5FieRvkH8XPG3,Contact,Pharmakon,0.1861,0.1513,0.3936,0.5853,0.2665,-10.4395,0.0721,...,1944130,0,Pharmakon,0,March 30 2017,Experimental,Each track of the new Pharmakon LP can be hear...,7.9000,underrated,52.3329
620,spotify:album:4MzrYQBl4zr5wzqnnHqWd4,All My Circles Run,Sarah Davachi,0.8336,0.1974,0.1384,0.8892,0.0975,-20.3378,0.0456,...,2486180,0,Sarah Davachi,0,March 16 2017,Electronic,Ambient composer Sarah Davachi departs from he...,7.3000,underrated,54.1681


High Energy Pitchfork Album Recommendations:

In [14]:
# generate vector based on user library
user_high_energy_dict = get_library_vector(user_df, 'energy', user_high_energy_thres, True, True)
print(user_high_energy_dict)

# get closest matches from pitchfork album reviews
user_recs_p4k_high_energy = P4K_Recommended('energy', user_high_energy_thres, True, user_high_energy_dict, sp_p4k_2017_df)
user_recs_p4k_high_energy.head(10)

{'popularity': {'average_value': 26.606060606060606, 'weight': 7.844186217881098}, 'danceability': {'average_value': 0.4903424242424242, 'weight': 16.33021022445993}, 'loudness': {'average_value': -6.898909090909086, 'weight': 14.127382784852403}, 'speechiness': {'average_value': 0.0805252525252525, 'weight': 5.6459396293041575}, 'acousticness': {'average_value': 0.13018249161616163, 'weight': 3.5784550941553124}, 'instrumentalness': {'average_value': 0.4587272910101012, 'weight': 6.923380729839011}, 'liveness': {'average_value': 0.23641717171717172, 'weight': 6.964001230297261}, 'valence': {'average_value': 0.42053232323232315, 'weight': 10.648055265193863}, 'tempo': {'average_value': 131.51369696969704, 'weight': 27.938388824016965}}


,album_uri,album,album_artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,duration_ms,is_explicit,artist,best,date,genre,review,score,group_label,distance
500,spotify:album:3Z9dDwHYs6mPAW2BHoT4JZ,Material Control,Glassjaw,0.0236,0.3493,0.8792,0.4644,0.2894,-5.3295,0.1488,...,2200421,0,Glassjaw,0,December 6 2017,Rock,"On their first album in 15 years, the Long Isl...",6.6000,overrated,8.3364
295,spotify:album:29HIscCMZKDTvGBWOQgbyf,"Hammersmith Odeon, London ’75",Bruce Springsteen and the E Street Band,0.1747,0.2926,0.7944,0.0077,0.8042,-6.7714,0.1496,...,7491795,0,Bruce Springsteen and the E Street Band,0,April 25 2017,Rock,"Reissued for Record Store Day, the live *Hamme...",9.2000,favorites,8.4646
271,spotify:album:20QC9JeWIAgKxPdTqI0A0U,Dedicated to Bobby Jameson,Ariel Pink,0.1198,0.4964,0.7659,0.0895,0.2335,-4.6739,0.0869,...,2981786,0,Ariel Pink,1,September 15 2017,Rock,Best new music Refining the gonzo pop-collages...,8.2000,favorites,8.9446
286,spotify:album:26hqnB0XF1ZFc31zY6NAgf,Near to the Wild Heart of Life,Japandroids,0.0269,0.2970,0.9359,0.0990,0.1544,-4.5505,0.0714,...,2208704,0,Japandroids,0,January 24 2017,Rock,"For the most part, Near to the Wild Heart of L...",7.1000,overrated,9.2550
306,spotify:album:2HJ2jjZjPe7jhKL6OwtoZY,GT Ultra,Guerilla Toss,0.0035,0.5397,0.8940,0.0498,0.1553,-4.6412,0.0636,...,1741672,0,Guerilla Toss,0,July 5 2017,Experimental,"On their latest, this Brooklyn dance-noise qui...",7.5000,favorites,11.0739
1,spotify:album:00MAXeszCotk3g9q8KYJlZ,Elastica,Elastica,0.0132,0.5434,0.7989,0.3546,0.1419,-5.6124,0.0444,...,2186449,1,Elastica,0,September 24 2017,Rock,"With its pithy, searing songs about sex, group...",8.5000,favorites,12.0764
832,spotify:album:5ypM6np3JwpXdApOm8bpQ8,Black Laden Crown,Danzig,0.0059,0.3591,0.8073,0.0138,0.1213,-7.6010,0.0519,...,2754211,2,Danzig,0,May 30 2017,Metal,Black Laden Crown is Danzig’s strongest album ...,6.4000,ignorables,12.4346
391,spotify:album:2pzI39O9jAVFghND0SiQHm,Who Built the Moon?,Noel Gallagher's High Flying Birds,0.1427,0.4050,0.7965,0.3102,0.2421,-5.4043,0.0494,...,2932568,0,Noel Gallagher's High Flying Birds,0,November 28 2017,Rock,Noel Gallagher’s latest is the most ambitious ...,7.1000,overrated,14.7982
632,spotify:album:4TDs45eAaIN56dI2FgZObB,Need to Feel Your Love,Sheer Mag,0.1788,0.6068,0.8212,0.3442,0.2171,-4.9954,0.0582,...,2560727,0,Sheer Mag,0,July 14 2017,Rock,"On their debut LP, Sheer Mag keep biting at th...",8.0000,favorites,15.8852
847,spotify:album:68aZtDbgNargCMYgNvw5JG,Ne? Wax Bloom,Iglooghost,0.0720,0.5215,0.7862,0.2446,0.3313,-8.1091,0.1214,...,2465594,0,Iglooghost,0,October 4 2017,Electronic,The new album from Irish producer Iglooghost p...,7.3000,favorites,17.0833


Low Energy Pitchfork Album Recommendations:

In [15]:
# generate vector based on user library
user_low_energy_dict = get_library_vector(user_df, 'energy', user_low_energy_thres, False, True)
print(user_low_energy_dict)

# get closest matches from pitchfork album reviews
user_recs_p4k_low_energy = P4K_Recommended('energy', user_low_energy_thres, False, user_low_energy_dict, sp_p4k_2017_df)
user_recs_p4k_low_energy.head(10)

{'popularity': {'average_value': 27.51, 'weight': 7.481536958587028}, 'danceability': {'average_value': 0.34167200000000003, 'weight': 9.709189153321725}, 'loudness': {'average_value': -17.527100000000004, 'weight': 17.231053125028758}, 'speechiness': {'average_value': 0.04074899999999999, 'weight': 10.042035799610774}, 'acousticness': {'average_value': 0.8299510000000002, 'weight': 18.826652584600158}, 'instrumentalness': {'average_value': 0.5886164716999999, 'weight': 7.542291179025434}, 'liveness': {'average_value': 0.13015800000000008, 'weight': 8.076335951408959}, 'valence': {'average_value': 0.17324799999999993, 'weight': 4.490504762127428}, 'tempo': {'average_value': 107.58671000000002, 'weight': 16.600400486289733}}


,album_uri,album,album_artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,duration_ms,is_explicit,artist,best,date,genre,review,score,group_label,distance
486,spotify:album:3UirGXBKa2VVuip8JPlBpd,Living Water,Shannon Lay,0.9394,0.4768,0.1349,0.3870,0.1128,-17.2567,0.0341,...,1723668,0,Shannon Lay,0,September 25 2017,Folk/Country,L.A. singer-songwriter Shannon Lay’s new album...,7.8000,underrated,14.6186
332,spotify:album:2SPoNPcAlfhUe84DH4SbQM,"Ripe Dreams, Pipe Dreams",Cameron Avery,0.4465,0.4692,0.3301,0.0740,0.1731,-17.4776,0.0862,...,3101082,0,Cameron Avery,0,March 25 2017,Rock,Tame Impala bassist Cameron Avery veers away f...,6.5000,overrated,20.4533
815,spotify:album:5n5V1pcevIOdJBF4xfuMlX,Finding People EP,Croatian Amor,0.8590,0.2340,0.2640,0.8780,0.0755,-16.5870,0.0347,...,202264,0,Croatian Amor,0,May 10 2017,Electronic,"On his new EP, Danish electronic artist Loke R...",6.3000,ignorables,24.7530
894,spotify:album:6Urg55LAGNL5QKM0bz8g2v,If Blue Could Be Happiness,Florist,0.9140,0.4672,0.1268,0.4061,0.1264,-19.7272,0.0395,...,2142776,0,Florist,0,October 2 2017,Folk/Country,The second full-length from New York folk-pop ...,7.7000,favorites,26.5202
662,spotify:album:4fo551Vy3KXbbRxRlVTD9D,Three Worlds: Music From Woolf Works,Max Richter,0.6855,0.4070,0.2744,0.7697,0.1583,-22.5071,0.0866,...,3979911,0,Max Richter,0,January 30 2017,Experimental,Max Richter's score for the narrative dance pi...,7.4000,favorites,26.7862
1017,spotify:album:7bxZ0kvGIbJRvBGFrKooub,Índia,Gal Costa,0.6098,0.6139,0.2940,0.0535,0.1288,-16.3557,0.0437,...,2318265,0,Gal Costa,0,July 26 2017,Pop/R&B,"With 1973’s Índia, the samba singer Gal Costa ...",8.5000,favorites,27.2091
126,spotify:album:0yf8QgcSSUM9x2CRRtalwL,Grunewald,Peter Broderick,0.9792,0.2760,0.0760,0.6581,0.0969,-20.4394,0.0450,...,1381786,0,Peter Broderick,0,December 16 2016,Experimental,Composer Peter Broderick manages to make simpl...,6.7000,ignorables,27.9257
818,spotify:album:5p64XgvFREt1P6mC7Xl6XN,A Crow Looked At Me,Mount Eerie,0.8588,0.5212,0.1059,0.0638,0.1315,-20.8825,0.0517,...,2494183,1,Mount Eerie,1,March 24 2017,Rock,Best new music Phil Elverum lost his wife—an a...,9.0000,favorites,28.2214
418,spotify:album:30b7gEVQNhrs74WkZl8Qo3,Monk's Music,Thelonious Monk,0.8163,0.5038,0.1818,0.2956,0.1118,-16.9983,0.0525,...,3715423,0,Thelonious Monk,0,March 12 2017,Jazz,"Featuring John Coltrane, Art Blakey, and Colem...",9.1000,underrated,28.4428
890,spotify:album:6U8MP9A9MF2fTdFRf0RTr1,Wildly Idle (Humble Before the Void),Hand Habits,0.4971,0.4186,0.3042,0.3286,0.1647,-13.7312,0.0341,...,2870705,0,Hand Habits,0,February 15 2017,Rock,The debut album by Hand Habits—aka Albany D.I....,7.6000,underrated,28.5277


## General Executable Cell:

In [18]:
# can stratify on any audio feature. We're focusing on Valence, Energy for paper. But see below for popularity.
audio_feat_sort = 'popularity'
audio_feat_threshold = user_df[audio_feat_sort].quantile(0.20) # get bottom 20% popularity
is_above = False # we're looking below the threshold, not above

# we want our library vector to be weighted by the standard deviation. It will treat smaller std deviations as more important audio features when searching
is_weighted = True 

# get our library vector
user_library_vector_dict = get_library_vector(user_df, audio_feat_sort, audio_feat_threshold, is_above, is_weighted)

# get our recommendations
user_recs_p4k = P4K_Recommended(audio_feat_sort, audio_feat_threshold, is_above, user_library_vector_dict, sp_p4k_2017_df)
user_recs_p4k.head(10)

# we will get my low popularity recommendations

,album_uri,album,album_artist,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,duration_ms,is_explicit,artist,best,date,genre,review,score,group_label,distance
972,spotify:album:71MtEZFFHlbjnoHVzkPhkM,Last Chance,House of Feelings,0.1010,0.6663,0.5250,0.0521,0.1237,-9.5269,0.0856,...,1902248,1,House of Feelings,0,August 14 2017,Electronic,House of Feelings make euphoric New York house...,6.9000,ignorables,3.5406
623,spotify:album:4OD0x9TEfpiXr9n5gEqDPn,The Golden Mile EP,Flash Trading,0.0266,0.6305,0.5782,0.4140,0.1061,-9.5535,0.0590,...,1489477,0,Flash Trading,0,August 8 2017,Electronic,Inspired by Brooklyn’s current electronic scen...,7.0000,ignorables,3.6298
452,spotify:album:3GEtLQSRAX7QUjcAD2puTd,Dribble EP,Sau Poler,0.2752,0.7167,0.7497,0.9327,0.0801,-11.1183,0.0393,...,1119791,0,Sau Poler,0,July 10 2017,Electronic,The Catalan producer Sau Poler puts an elegant...,7.8000,underrated,5.4267
141,spotify:album:15jLsbz7C3DMzRagWPtHeW,U Feel Anything?,Ziúr,0.0370,0.4664,0.6169,0.6375,0.1274,-11.2304,0.0822,...,2408130,0,Ziúr,0,October 7 2017,Electronic,"In her abraded experimental club music, Berlin...",7.1000,ignorables,6.9128
642,spotify:album:4WaEM0WzThlJ1ZxGHsxqug,Rules,Annie Hardy,0.3784,0.3911,0.4314,0.1170,0.2226,-8.5546,0.0328,...,3335653,0,Annie Hardy,0,April 10 2017,Rock,"On her first solo album, Giant Drag singer Ann...",7.3000,underrated,7.0145
704,spotify:album:53kAh5L9h1fVuiC9G0SIOT,Hey Mr Ferryman,Mark Eitzel,0.3419,0.4686,0.4595,0.0255,0.1542,-11.0088,0.0392,...,3121778,0,Mark Eitzel,0,January 26 2017,Rock,Hey Mr Ferryman is Mark Eitzel’s heaviest albu...,8.1000,underrated,7.9012
812,spotify:album:5m66mMFCaALe0TzWBxrsvq,p h a r a o h,Jansport J,0.1413,0.5488,0.7630,0.4769,0.3172,-8.2696,0.2942,...,2422247,0,Jansport J,0,January 28 2017,Rap,California producer Jansport J makes instrumen...,7.6000,underrated,9.7337
95,spotify:album:0leXjA4V9EBfZCesis2G7w,Opposites,Parlor Walls,0.4561,0.3168,0.7479,0.4584,0.1435,-8.5795,0.1120,...,2373964,0,Parlor Walls,0,March 15 2017,Rock,Brooklyn trio Parlor Walls subvert the dichoto...,7.5000,underrated,10.4325
102,spotify:album:0pq6OVUmtM6HqdqJDDc6q0,Siberia,Vanbot,0.2091,0.4690,0.6129,0.7149,0.2817,-9.7464,0.0489,...,3428355,0,Vanbot,0,April 12 2017,Electronic,The Swedish electronic artist Vanbot and two c...,6.0000,ignorables,10.4854
978,spotify:album:73MViyQgUqY6WY7Tm9A7Ps,New Shapes of Life,Martin Carr,0.2299,0.5149,0.5701,0.1563,0.1702,-9.2432,0.0311,...,1855851,0,Martin Carr,0,November 16 2017,Rock,The Boo Radleys mastermind returns with a suav...,7.4000,underrated,12.4219


## Plots and Analysis of our recommendations below!!!